<a href="https://colab.research.google.com/github/jordhanus/StarWars/blob/main/RLforRaid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
from gym import spaces
import numpy as np
import random

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class SWGoHEnv(gym.Env):
    def __init__(self):
        super(SWGoHEnv, self).__init__()
        self.action_space = spaces.Discrete(3)  # 0: Basic Attack, 1: Special 1, 2: Special 2
        self.observation_space = spaces.Box(low=0, high=100, shape=(6,), dtype=np.int)

        # Game state: [Toon1 HP, Toon2 HP, Boss HP, Special1 CD, Special2 CD, Boss Enrage]
        self.state = np.array([20, 20, 100, 0, 0, 0])
        self.done = False

    def step(self, action):
        reward = 0
        toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage = self.state

        # Player action
        if action == 0:  # Basic Attack
            boss_hp -= 2
            boss_enrage += 2  # Increase enrage by 2%
        elif action == 1 and special1_cd == 0:  # Special 1
            boss_hp -= 10
            boss_enrage += 2  # Increase enrage by 2% for any damage
            special1_cd = 4
        elif action == 2 and special2_cd == 0:  # Special 2
            toon1_hp = min(20, toon1_hp + 10)
            toon2_hp = min(20, toon2_hp + 10)
            special2_cd = 4

        # Update cooldowns
        special1_cd = max(0, special1_cd - 1)
        special2_cd = max(0, special2_cd - 1)

        # Boss actions (2 turns)
        for _ in range(2):
            if boss_enrage >= 100:
                reward = -100  # Penalty for boss reaching full enrage
                self.done = True
                break

            if boss_hp <= 0:
                reward = 100  # Reward for defeating the boss
                self.done = True
                break

            # Boss basic attack
            target_hp = toon1_hp if toon1_hp < toon2_hp else toon2_hp
            target_hp -= 10

            # Update the HPs
            if toon1_hp < toon2_hp:
                toon1_hp = target_hp
            else:
                toon2_hp = target_hp

            # Check for defeat conditions
            if toon1_hp <= 0 or toon2_hp <= 0:
                reward = -100  # Penalty for losing a toon
                self.done = True
                break

        self.state = np.array([toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage])
        return self.state, reward, self.done, {}

    def reset(self):
        self.state = np.array([20, 20, 100, 0, 0, 0])
        self.done = False
        return self.state

    def render(self, mode='human'):
        print(f"State: {self.state}")


In [ ]:
class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.995):
        self.action_space = action_space
        self.q_table = np.zeros([100, 100, 100, 5, 5, 100, action_space.n])  # Adjust the dimensions based on your state space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay

    def choose_action(self, state):
        if random.uniform(0, 1) < self.exploration_rate:
            return self.action_space.sample()  # Explore action space
        else:
            return np.argmax(self.q_table[state])  # Exploit learned values

    def learn(self, state, action, reward, next_state, done):
        old_value = self.q_table[state][action]
        next_max = np.max(self.q_table[next_state])

        new_value = (1 - self.learning_rate) * old_value + self.learning_rate * (reward + self.discount_factor * next_max)
        self.q_table[state][action] = new_value

        if done:
            self.exploration_rate *= self.exploration_decay


In [ ]:
env = SWGoHEnv()
agent = QLearningAgent(env.action_space)

num_episodes = 1000  # Total number of training episodes
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward

    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}, Exploration Rate: {agent.exploration_rate}")


<ipython-input-9-257d013e2393>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.observation_space = spaces.Box(low=0, high=100, shape=(6,), dtype=np.int)


IndexError: ignored

In [ ]:
import gym
from gym import spaces
import numpy as np
import random

class UpdatedSWGoHEnv(gym.Env):
    def __init__(self):
        super(UpdatedSWGoHEnv, self).__init__()
        self.action_space = spaces.Discrete(3)  # 0: Basic Attack, 1: Special 1, 2: Special 2
        self.observation_space = spaces.Box(low=0, high=100, shape=(6,), dtype=int)

        # Game state: [Toon1 HP, Toon2 HP, Boss HP, Special1 CD, Special2 CD, Boss Enrage]
        self.state = np.array([50, 50, 50, 0, 0, 0])
        self.done = False

    def step(self, action):
        reward = 0
        toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage = self.state

        # Player action
        if action == 0:  # Basic Attack
            boss_hp = max(0, boss_hp - 5)
            boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2%
        elif action == 1 and special1_cd == 0:  # Special 1
            boss_hp = max(0, boss_hp - 40)
            boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2% for any damage
            special1_cd = 4
        elif action == 2 and special2_cd == 0:  # Special 2
            toon1_hp = min(20, toon1_hp + 10)
            toon2_hp = min(20, toon2_hp + 10)
            special2_cd = 2

        # Update cooldowns
        special1_cd = max(0, special1_cd - 1)
        special2_cd = max(0, special2_cd - 1)

        # Boss actions (2 turns)
        for _ in range(2):
            if boss_enrage >= 100:
                reward = -100  # Penalty for boss reaching full enrage
                self.done = True
                break

            if boss_hp <= 0:
                reward = 100  # Reward for defeating the boss
                self.done = True
                break

            # Boss basic attack
            target_hp = toon1_hp if toon1_hp < toon2_hp else toon2_hp
            target_hp = max(0, target_hp - 10)

            # Update the HPs
            if toon1_hp < toon2_hp:
                toon1_hp = target_hp
            else:
                toon2_hp = target_hp

            # Check for defeat conditions
            if toon1_hp <= 0 or toon2_hp <= 0:
                reward = -100  # Penalty for losing a toon
                self.done = True
                break

        self.state = np.array([toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage])
        return self.state, reward, self.done, {}

    def reset(self):
        self.state = np.array([20, 20, 100, 0, 0, 0])
        self.done = False
        return self.state

    def render(self, mode='human'):
        print(f"State: {self.state}")

def get_state_index(state):
    base = [21, 21, 51, 5, 5, 101]
    index = 0
    multiplier = 1
    for s, b in zip(state, base):
        index += s * multiplier
        multiplier *= b
    return index

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.995):
        self.action_space = action_space
        total_states = 21 * 21 * 101 * 5 * 5 * 101  # Based on the state index calculation
        self.q_table = np.zeros([total_states, action_space.n])
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay

    def choose_action(self, state):
        state_index = get_state_index(state)
        if random.uniform(0, 1) < self.exploration_rate:
            return self.action_space.sample()
        else:
            return np.argmax(self.q_table[state_index])

    def learn(self, state, action, reward, next_state, done):
        state_index = get_state_index(state)
        next_state_index = get_state_index(next_state)
        old_value = self.q_table[state_index][action]
        next_max = np.max(self.q_table[next_state_index])

        new_value = (1 - self.learning_rate) * old_value + self.learning_rate * (reward + self.discount_factor * next_max)
        self.q_table[state_index][action] = new_value

        if done:
            self.exploration_rate *= self.exploration_decay

env = UpdatedSWGoHEnv()
agent = QLearningAgent(env.action_space)

num_episodes = 10000  # Total number of training episodes
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward

    if episode % 1000 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}, Exploration Rate: {agent.exploration_rate}")


Episode: 0, Total Reward: -100, Exploration Rate: 0.995
Episode: 1000, Total Reward: -100, Exploration Rate: 0.0066206987359377885
Episode: 2000, Total Reward: -100, Exploration Rate: 4.405392135884259e-05
Episode: 3000, Total Reward: -100, Exploration Rate: 2.931334084960731e-07
Episode: 4000, Total Reward: -100, Exploration Rate: 1.9505004895387866e-09
Episode: 5000, Total Reward: -100, Exploration Rate: 1.2978568970387459e-11
Episode: 6000, Total Reward: -100, Exploration Rate: 8.635899012716164e-14
Episode: 7000, Total Reward: -100, Exploration Rate: 5.746300068057951e-16
Episode: 8000, Total Reward: -100, Exploration Rate: 3.823570009739748e-18
Episode: 9000, Total Reward: -100, Exploration Rate: 2.5441914703772496e-20


In [ ]:
import gym
from gym import spaces
import numpy as np
import random

class UpdatedSWGoHEnv(gym.Env):
    def __init__(self):
        super(UpdatedSWGoHEnv, self).__init__()
        self.action_space = spaces.Discrete(6)  # 3 actions per toon
        self.observation_space = spaces.Box(low=0, high=100, shape=(8,), dtype=int)

        # State: [Toon1 HP, Toon2 HP, Boss HP, Toon1 Special1 CD, Toon1 Special2 CD, Toon2 Special1 CD, Toon2 Special2 CD, Boss Enrage]
        self.state = np.array([20, 20, 100, 0, 0, 0, 0, 0])
        self.done = False

    def step(self, action):
        reward = 0
        toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage = self.state

        # Player action
        if action == 0:  # Basic Attack
            boss_hp = max(0, boss_hp - 5)
            boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2%
        elif action == 1 and special1_cd == 0:  # Special 1
            boss_hp = max(0, boss_hp - 40)
            boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2% for any damage
            special1_cd = 4
        elif action == 2 and special2_cd == 0:  # Special 2
            toon1_hp = min(20, toon1_hp + 10)
            toon2_hp = min(20, toon2_hp + 10)
            special2_cd = 2

        # Update cooldowns
        special1_cd = max(0, special1_cd - 1)
        special2_cd = max(0, special2_cd - 1)

        # Boss actions (2 turns)
        for _ in range(2):
            if boss_enrage >= 100:
                reward = -100  # Penalty for boss reaching full enrage
                self.done = True
                break

            if boss_hp <= 0:
                reward = 100  # Reward for defeating the boss
                self.done = True
                break

            # Boss basic attack
            target_hp = toon1_hp if toon1_hp < toon2_hp else toon2_hp
            target_hp = max(0, target_hp - 10)

            # Update the HPs
            if toon1_hp < toon2_hp:
                toon1_hp = target_hp
            else:
                toon2_hp = target_hp

            # Check for defeat conditions
            if toon1_hp <= 0 or toon2_hp <= 0:
                reward = -100  # Penalty for losing a toon
                self.done = True
                break

        self.state = np.array([toon1_hp, toon2_hp, boss_hp, special1_cd, special2_cd, boss_enrage])
        return self.state, reward, self.done, {}

    def reset(self):
        self.state = np.array([20, 20, 100, 0, 0, 0, 0, 0])
        self.done = False
        return self.state

    def render(self, mode='human'):
        print(f"State: {self.state}")

def get_state_index(state):
    base = [21, 21, 101, 5, 5, 5, 5, 101]
    index = 0
    multiplier = 1
    for s, b in zip(state, base):
        index += s * multiplier
        multiplier *= b
    return index

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.995):
        self.action_space = action_space
        total_states = 21 * 21 * 101 * 5 * 5 * 5 * 5 * 101
        self.q_table = np.zeros([total_states, action_space.n])
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay

    def choose_action(self, state):
        state_index = get_state_index(state)
        if random.uniform(0, 1) < self.exploration_rate:
            return self.action_space.sample()
        else:
            return np.argmax(self.q_table[state_index])

    def learn(self, state, action, reward, next_state, done):
        state_index = get_state_index(state)
        next_state_index = get_state_index(next_state)
        old_value = self.q_table[state_index][action]
        next_max = np.max(self.q_table[next_state_index])

        new_value = (1 - self.learning_rate) * old_value + self.learning_rate * (reward + self.discount_factor * next_max)
        self.q_table[state_index][action] = new_value

        if done:
            self.exploration_rate *= self.exploration_decay

env = UpdatedSWGoHEnv()
agent = QLearningAgent(env.action_space)

num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward

    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}, Exploration Rate: {agent.exploration_rate}")


ValueError: ignored

In [ ]:
import gym
from gym import spaces
import numpy as np
import random

class UpdatedSWGoHEnv(gym.Env):
    def __init__(self):
        super(UpdatedSWGoHEnv, self).__init__()
        self.action_space = spaces.Discrete(6)  # 3 actions per toon
        self.observation_space = spaces.Box(low=0, high=100, shape=(8,), dtype=int)

        # State: [Toon1 HP, Toon2 HP, Boss HP, Toon1 Special1 CD, Toon1 Special2 CD, Toon2 Special1 CD, Toon2 Special2 CD, Boss Enrage]
        self.state = np.array([20, 20, 50, 0, 0, 0, 0, 0])
        self.done = False

    def step(self, action):
        reward = 0
        toon1_hp, toon2_hp, boss_hp, t1_s1_cd, t1_s2_cd, t2_s1_cd, t2_s2_cd, boss_enrage = self.state

        # Determine which toon is acting and which action is taken
        toon_acting = 1 if action < 3 else 2
        action = action % 3

        # Player action
        if action == 0:  # Basic Attack
            boss_hp = max(0, boss_hp - 10)
            boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2%
        elif action == 1:  # Special 1
            if (toon_acting == 1 and t1_s1_cd == 0) or (toon_acting == 2 and t2_s1_cd == 0):
                boss_hp = max(0, boss_hp - 20)
                boss_enrage = min(100, boss_enrage + 2)  # Increase enrage by 2%
                if toon_acting == 1:
                    t1_s1_cd = 4
                else:
                    t2_s1_cd = 4
        elif action == 2:  # Special 2
            if (toon_acting == 1 and t1_s2_cd == 0) or (toon_acting == 2 and t2_s2_cd == 0):
                toon1_hp = min(20, toon1_hp + 20)
                toon2_hp = min(20, toon2_hp + 20)
                if toon_acting == 1:
                    t1_s2_cd = 4
                else:
                    t2_s2_cd = 4

        # Update cooldowns
        t1_s1_cd = max(0, t1_s1_cd - 1)
        t1_s2_cd = max(0, t1_s2_cd - 1)
        t2_s1_cd = max(0, t2_s1_cd - 1)
        t2_s2_cd = max(0, t2_s2_cd - 1)

        # Boss actions (2 turns)
        for _ in range(2):
            if boss_enrage >= 100:
                reward = -100  # Penalty for boss reaching full enrage
                self.done = True
                break

            if boss_hp <= 0:
                reward = 100  # Reward for defeating the boss
                self.done = True
                break

            # Boss basic attack
            target_hp = toon1_hp if toon1_hp < toon2_hp else toon2_hp
            target_hp = max(0, target_hp - 5)

            # Update the HPs
            if toon1_hp < toon2_hp:
                toon1_hp = target_hp
            else:
                toon2_hp = target_hp

            # Check for defeat conditions
            if toon1_hp <= 0 or toon2_hp <= 0:
                reward = -50  # Penalty for losing a toon
                # self.done = True
                # break

        self.state = np.array([toon1_hp, toon2_hp, boss_hp, t1_s1_cd, t1_s2_cd, t2_s1_cd, t2_s2_cd, boss_enrage])
        return self.state, reward, self.done, {}

    def reset(self):
        self.state = np.array([20, 20, 100, 0, 0, 0, 0, 0])
        self.done = False
        return self.state

    def render(self, mode='human'):
        print(f"State: {self.state}")

def get_state_index(state):
    base = [21, 21, 101, 5, 5, 5, 5, 101]
    index = 0
    multiplier = 1
    for s, b in zip(state, base):
        index += s * multiplier
        multiplier *= b
    return index

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.995):
        self.action_space = action_space
        total_states = 21 * 21 * 101 * 5 * 5 * 5 * 5 * 101
        self.q_table = np.zeros([total_states, action_space.n])
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay

    def choose_action(self, state):
        state_index = get_state_index(state)
        if random.uniform(0, 1) < self.exploration_rate:
            return self.action_space.sample()
        else:
            return np.argmax(self.q_table[state_index])

    def learn(self, state, action, reward, next_state, done):
        state_index = get_state_index(state)
        next_state_index = get_state_index(next_state)
        old_value = self.q_table[state_index][action]
        next_max = np.max(self.q_table[next_state_index])

        new_value = (1 - self.learning_rate) * old_value + self.learning_rate * (reward + self.discount_factor * next_max)
        self.q_table[state_index][action] = new_value

        if done:
            self.exploration_rate *= self.exploration_decay

env = UpdatedSWGoHEnv()
agent = QLearningAgent(env.action_space, learning_rate=0.01, discount_factor=0.99, exploration_rate=2.0, exploration_decay=0.995)

best_score = float('-inf')
best_actions = []
best_episode = 0

num_episodes = 10000
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    episode_actions = []

    while not done:
        action = agent.choose_action(state)
        episode_actions.append(action)  # Record action
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward

    if total_reward > best_score:
        best_score = total_reward
        best_actions = episode_actions
        best_episode = episode

    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}, Exploration Rate: {agent.exploration_rate}")

print(f"Best Episode: {best_episode}, Best Score: {best_score}, Actions: {best_actions}")

Episode: 0, Total Reward: -400, Exploration Rate: 1.99
Episode: 100, Total Reward: -350, Exploration Rate: 1.2054831686165484
Episode: 200, Total Reward: -300, Exploration Rate: 0.7302460652350725
Episode: 300, Total Reward: -150, Exploration Rate: 0.4423614776830866
Episode: 400, Total Reward: -200, Exploration Rate: 0.2679695054227667
Episode: 500, Total Reward: -200, Exploration Rate: 0.1623280042661538
Episode: 600, Total Reward: -250, Exploration Rate: 0.09833350599897663
Episode: 700, Total Reward: -150, Exploration Rate: 0.05956753085066369
Episode: 800, Total Reward: -250, Exploration Rate: 0.036084249164081414
Episode: 900, Total Reward: -300, Exploration Rate: 0.021858771366565784
Episode: 1000, Total Reward: -300, Exploration Rate: 0.013241397471875577
Episode: 1100, Total Reward: -550, Exploration Rate: 0.008021247126285285
Episode: 1200, Total Reward: -150, Exploration Rate: 0.004859034372889833
Episode: 1300, Total Reward: -300, Exploration Rate: 0.002943459372988924
Epis